# Lab 04 - Acquisition functions and Bayesian optimization
## Tasks
- Demonstrate Bayesian optimization
- Solve quadrupole triplet focusing using Bayesian optimization

# Set up environment

In [ ]:
%reset -f
!pip install botorch==0.12.0 gpytorch xopt==2.5.2
!pip install cheetah-accelerator

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

#import packages required for BO
import torch

from botorch.models import SingleTaskGP
from botorch.models.transforms import Normalize, Standardize
from botorch.fit import fit_gpytorch_mll
from botorch.acquisition import LogExpectedImprovement
from gpytorch.mlls import ExactMarginalLogLikelihood

## GP model creation
We start by generating two random observations to create the initial GP model (using BoTorch this time). Note that BoTorch does a number of things under the hood, including using Normalization and Standardization transformers to normalize and standardize the data.

In [ ]:
def f(x):
    return torch.sin(2*torch.pi*x) + x + torch.randn_like(x) * 0.01

train_x = torch.tensor([0.3,0.5],dtype=torch.double).unsqueeze(-1)
train_y = f(train_x)

gp = SingleTaskGP(
    train_X=train_x,
    train_Y=train_y,
    input_transform=Normalize(d=1),
    outcome_transform=Standardize(m=1),
)
mll = ExactMarginalLogLikelihood(gp.likelihood, gp)
fit_gpytorch_mll(mll);

### Acquisition function definition
Next we define the acquisition function. For this example we use Log Expected Improvement (EI).

In [ ]:
acquisition_function = LogExpectedImprovement(gp, best_f=torch.max(train_y))

### Visualize the GP model and the acquisition function.

In [ ]:
x = torch.linspace(0, 1, 50)
with torch.no_grad():
    p = gp.posterior(x.reshape(-1,1,1))

    #get the mean
    m = p.mean.squeeze()

    #get the 2 sigma confidence region around the mean
    l,u = p.mvn.confidence_region()
    l = l.squeeze()
    u = u.squeeze()

    # calculate the acquisition function
    acqf = acquisition_function(x.reshape(-1,1,1))


fig,ax = plt.subplots(2,1,sharex=True)
ax[1].set_xlabel('x')
ax[0].set_ylabel('y')
#plot mean and confidence region
ax[0].plot(x, m)
ax[0].fill_between(x.squeeze(), l, u, alpha = 0.25, lw = 0)

ax[1].plot(x, acqf)
ax[1].set_ylabel(r"$\alpha(x)$")


### Optimize the acquisition function
Use the `optimize_acqf` function in BoTorch to maximize the acquisition function.

In [ ]:
from botorch.optim import optimize_acqf

bounds = torch.stack([torch.zeros(1), torch.ones(1)]).to(torch.double)
candidate, acq_value = optimize_acqf(
    acquisition_function, bounds=bounds, q=1, num_restarts=5, raw_samples=20,
)
candidate

# Basic BO
We are going to maximize the following function using Bayesian optimization

$$
f(x) = \sin(2\pi x) + x
$$
in the domain $[0,1]$.

### **Task**
Perform 20 steps of BO to solve the optimization problem. Plot the model and the acquisition function at the end of each optimization step.

# Trust region BO

## **Task**
Rewrite the optimization loop above to set the acquisition function bounds centered at the location of the current optimum +/- 10\% of the input domain. Plot the objective function vs iteration for both cases. How does this affect optimization performance?

In [ ]:
train_x = torch.tensor([0.5],dtype=torch.double).unsqueeze(-1)
train_y = f(train_x)

> **Your answer here** (How does this affect optimization performance?)

 # **Homework**
Minimize the beamsize function defined in lab 1, using Xopt's `ExpectedImprovementGenerator`, defined for you below. Note that you can call `X.random_evaluate()` to generate random samples for creating the initial GP model.
  See https://xopt.xopt.org/examples/single_objective_bayes_opt/bo_tutorial/ for
  an example.

In [ ]:
from xopt.generators.bayesian import ExpectedImprovementGenerator
from xopt import VOCS

vocs = VOCS(
    variables={
        "k1": [0,15],
        "k2": [-15,0],
        "k3": [0, 15]
    },
    objectives={"beamsize": "MINIMIZE"}
)

generator = ExpectedImprovementGenerator(vocs=vocs)

# **BONUS Homework (NOT GRADED)**
Now solve the multi-objective 2-D ZDT problem from Lab 2 using Mulit-Objective Bayesian Optimization (MOBO) using the `MOBOGenerator` object in Xopt (see https://xopt.xopt.org/examples/multi_objective_bayes_opt/mobo/ for an example). Plot the front projected onto the bunch length vs. horizontal emittance subspace. Plot the acquisition function on the 2D input plane every 5 steps. Finally, calculate the hypervolume of the Pareto front at the end of the optimization run.